In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
from pandas.io.formats.excel import ExcelFormatter
import os

In [24]:

# Geckodriver yolu
geckodriver_path = "/Users/yakupdemir/Desktop/mackolikData/drivers/geckodriver"

# Firefox service nesnesini oluştur
service = FirefoxService(executable_path=geckodriver_path)


# Firefox tarayıcısını service ile başlat
driver = webdriver.Firefox(service=service)
# Hedef web sayfasına git

macKolikUrl="https://www.mackolik.com/futbol/canli-sonuclar"
driver.get(macKolikUrl)

#time.sleep(5)
wait = WebDriverWait(driver, 30)
cookies=wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#didomi-notice-agree-button')))
cookies.click()

#click for ordering by matches start time
#clock=wait.until(EC.element_to_be_clickable((By.XPATH, "//span[.='Saat']")))
#clock.click()

dateIcon = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.widget-dateslider__datepicker-toggle')))

# Click on the element
dateIcon.click()
butunTakvim = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'.widget-datepicker')))
html_contentTakvim = butunTakvim.get_attribute('outerHTML')
# print(html_contentTakvim)



# BeautifulSoup ile HTML içeriğini parse et
soup = BeautifulSoup(html_contentTakvim, 'html.parser')

# Günü ve Yılı al
day = int(soup.find('td', {'class': 'widget-datepicker__calendar-body-cell--selected-day'}).text.strip())
year = int(soup.find('div', {'class': 'widget-datepicker__selector--year'}).find('div', {'class': 'widget-datepicker__value'}).text.strip())

#print(f"day: {day}, year: {year}")

current_year = int(driver.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__value').text)
previous_year_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker.widget-dateslider__datepicker--open > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__nav.widget-datepicker__nav--previous")))

# Hedef yılı belirle (örneğin, 2020)
target_year = 2022

# Hedef yıla kadar önceki yıl düğmesine tıkla
while current_year > target_year:
    #print(str(target_year)+ "target year")
    previous_year_button.click()
    #time.sleep(1)
    current_year -= 1
    #print(current_year)
    # Yeni yılı kontrol et
    current_year = int(driver.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__value').text)
    #print(current_year)

    
# calc of months
previous_month_button= wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker.widget-dateslider__datepicker--open > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__nav.widget-datepicker__nav--previous")))
#wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
current_month = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__value")))
month_str = current_month.text.strip()[:3]

#print(month_str)
# Map the month abbreviation to a numeric value
month_mapping = {'OCA': 1, 'ŞUB': 2, 'MAR': 3, 'NIS': 4, 'MAY': 5, 'HAZ': 6, 'TEM': 7, 'AĞU': 8, 'EYL': 9, 'EKI': 10, 'KAS': 11, 'ARA': 12}
#print(month_str+"month_str")
month_numeric = month_mapping.get(month_str)

target_month=12
month_numeric = month_mapping.get(month_str)


# Loop until you reach the target month
while month_numeric != target_month:
    #print(str(month_numeric)+"month_numeric")
    #print(str(target_month)+"target_month")

    
    previous_month_button.click()
    #time.sleep(1)
    
    current_month = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__value")))
    month_str = current_month.text.strip()[:3]
    #print(month_str+"month_str")
    month_numeric = month_mapping.get(month_str)
    #print(month_numeric)
    #print(target_month)
    
target_day=14
day_element = driver.find_element(By.XPATH, f"//td[contains(@class, 'widget-datepicker__calendar-body-cell') and text()='{target_day}']")
#time.sleep(19)                   #//td[contains(@class, 'widget-datepicker__calendar-body-cell') and text()='{target_day}']
day_element.click()

time.sleep(2)


# wait for page loaded
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, "//body"))
)

#daily all  matches rows container elements xpath
container_css = "div.match-row__match-content"  #(//div[@class='match-row__match-content'])[1]


wait_time = 15

#daily all  matches rows container ELEMENT
allLinksElements = WebDriverWait(driver, wait_time).until(
    EC.visibility_of_all_elements_located((By.CSS_SELECTOR, container_css))
)

# Container elementini bul (alternative)
#allLinksElements = driver.find_elements(By.XPATH, container_xpath)

#dataframe all bettings odds one match
#df_all = pd.DataFrame()
count = 0
iteration_count = 0
max_iterations = 1

while True:
    try:
        # Container elementini bul
        #allLinksElements = driver.find_elements(By.XPATH, container_css)

        # If all links are processed, break out of the loop
        if not allLinksElements:
            break
        restart_loop = False
        
        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=1):
            result=[]
            try:
                # Precautionary for page loading
                #dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                #elementx = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                print(f"(try Worked... {str(i)})")
                element.click()

                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                
                # Perform actions in the new tab
                iddiaButton = WebDriverWait(driver, wait_time).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-item-id='iddaa']"))
                )

                # get class attribute values...
                tx = iddiaButton.get_attribute('class')

                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    
                    # Top scoreboard table element
                    macScorboard = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "//div[@class='p0c-soccer-match-details-header']"))
                    )
                    
                    # Main betting table element
                    macDetailsData = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/main/div[1]/div[2]/div/div[2]/div"))
                    )
                    
                    # Scoreboard as a string
                    scoreboard_text = macScorboard.text
                    lines = scoreboard_text.strip().split('\n')
                    for line_sb in lines:
                        result += [line_sb]
                    #print(result)
                    
                    # Get match details as string text
                    mac_Details_Data = macDetailsData.text
                    detail_lines = mac_Details_Data.strip().split('\n')## detail_lines is a list
                    
                    for line_md in detail_lines:
                        result+=[line_md]
                        
                    #print(result)
                    
                    ham_df=pd.DataFrame(result, columns=[0])
                    #pd.options.display.max_rows = None
                    print(ham_df)
                    
                    
                    # directory = './dataSet'
                    # if not os.path.exists(directory):
                    #     os.makedirs(directory)

                    # DataFrame'i Excel dosyasına kaydet
                    ham_df.to_excel(f'../dataSet/new_data_raw_{i}_.xlsx')
                    
                    
                    # Print match details...
                    #print(detail_lines)                                            
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(1)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

            # Increment the iteration count
            iteration_count += 1

            # Check if it's time to restart the loop
            if iteration_count % max_iterations == 0:
                print(f"Restarting the loop after {max_iterations} iterations.")
                restart_loop=True
                break
                
                
        if restart_loop:
            break       

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue


(try Worked... 1)
(went new page... 1)
Button is not disabled. Proceeding...
                                                     0
0                                               Fransa
1                                                  Fas
2                                                    2
3                                                   MS
4                                                    0
5                                           (İY 1 - 0)
6    14-12-22 | Dünya Kupası | Stat: Al Bayt Stadiu...
7                                                 Tümü
8                                           Maç Sonucu
9                                         Devre Sonucu
10                                           Altı/Üstü
11                                          Çifte Şans
12                              MAÇ SONUCU 00892 MBS 1
13                                                   1
14                                                1.41
15                                         

In [ ]:
while True:
    try:
        # Container elementini bul
        #allLinksElements = driver.find_elements(By.XPATH, container_css)

        # If all links are processed, break out of the loop
        if not allLinksElements:
            break
        restart_loop = False
        
        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=4):
            try:
                # Precautionary for page loading
                dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                elementx = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                print(f"(try Worked... {str(i)})")
                elementx.click()

                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                
                # Perform actions in the new tab
                iddiaButton = WebDriverWait(driver, wait_time).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-item-id='iddaa']"))
                )

                # get class attribute values...
                tx = iddiaButton.get_attribute('class')

                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    
                    # Top scoreboard table element
                    macScorboard = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "//div[@class='p0c-soccer-match-details-header']"))
                    )
                    
                    # Main betting table element
                    macDetailsData = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/main/div[1]/div[2]/div/div[2]/div"))
                    )
                    
                    # Scoreboard as a string
                    scoreboard_text = macScorboard.text
                    lines = scoreboard_text.strip().split('\n')
                    print(lines)
                    
                    # Get match details as string text
                    mac_Details_Data = macDetailsData.text
                    detail_lines = mac_Details_Data.strip().split('\n')  # detail_line is a list
                    
                    for line_md in detail_lines:
                        result+=[line_md]
                    
                    
                    
                    
                    # Print match details...
                    #print(detail_lines)                                            
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(1)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

            # Increment the iteration count
            iteration_count += 1

            # Check if it's time to restart the loop
            if iteration_count % max_iterations == 0:
                print(f"Restarting the loop after {max_iterations} iterations.")
                restart_loop=True
                break
                
                
        if restart_loop:
            break       

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue

In [ ]:
while True:
    try:
        # Container elementini bul
        #allLinksElements = driver.find_elements(By.XPATH, container_css)

        # If all links are processed, break out of the loop
        if not allLinksElements:
            break
        restart_loop = False
        
        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=8):
            try:
                # Precautionary for page loading
                dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                elementx = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                print(f"(try Worked... {str(i)})")
                elementx.click()

                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                
                # Perform actions in the new tab
                iddiaButton = WebDriverWait(driver, wait_time).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-item-id='iddaa']"))
                )

                # get class attribute values...
                tx = iddiaButton.get_attribute('class')

                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    
                    # Top scoreboard table element
                    macScorboard = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "//div[@class='p0c-soccer-match-details-header']"))
                    )
                    
                    # Main betting table element
                    macDetailsData = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/main/div[1]/div[2]/div/div[2]/div"))
                    )
                    
                    # Scoreboard as a string
                    scoreboard_text = macScorboard.text
                    lines = scoreboard_text.strip().split('\n')
                    print(lines)
                    
                    # Get match details as string text
                    mac_Details_Data = macDetailsData.text
                    detail_lines = mac_Details_Data.strip().split('\n')
                    # Print match details...
                    print(detail_lines)                                            
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(1)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

            # Increment the iteration count
            iteration_count += 1

            # Check if it's time to restart the loop
            if iteration_count % max_iterations == 0:
                print(f"Restarting the loop after {max_iterations} iterations.")
                restart_loop=True
                break
                
                
        if restart_loop:
            break       

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue

In [ ]:
while True:
    try:
        # Container elementini bul
        #allLinksElements = driver.find_elements(By.XPATH, container_css)

        # If all links are processed, break out of the loop
        if not allLinksElements:
            break
        restart_loop = False
        
        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=12):
            try:
                # Precautionary for page loading
                dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                elementx = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                print(f"(try Worked... {str(i)})")
                elementx.click()

                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                
                # Perform actions in the new tab
                iddiaButton = WebDriverWait(driver, wait_time).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-item-id='iddaa']"))
                )

                # get class attribute values...
                tx = iddiaButton.get_attribute('class')

                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    
                    # Top scoreboard table element
                    macScorboard = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "//div[@class='p0c-soccer-match-details-header']"))
                    )
                    
                    # Main betting table element
                    macDetailsData = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/main/div[1]/div[2]/div/div[2]/div"))
                    )
                    
                    # Scoreboard as a string
                    scoreboard_text = macScorboard.text
                    lines = scoreboard_text.strip().split('\n')
                    print(lines)
                    
                    # Get match details as string text
                    mac_Details_Data = macDetailsData.text
                    detail_lines = mac_Details_Data.strip().split('\n')
                    # Print match details...
                    print(detail_lines)                                            
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(1)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

            # Increment the iteration count
            iteration_count += 1

            # Check if it's time to restart the loop
            if iteration_count % max_iterations == 0:
                print(f"Restarting the loop after {max_iterations} iterations.")
                restart_loop=True
                break
                
                
        if restart_loop:
            break

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue
        
driver.quit()        